# Joining files together

The purpose of this file is to convey methodology and logic that can be used to join our files together.  To this end, we will cover each our of Raw 2.0 files using a slice of the full job records.  These methodologies can be scaled up in a SQL data warehouse or by chunking the files depending on the subset you are working with.

In [1]:
import pandas as pd
import tarfile
import sqlite3
from xml.etree import ElementTree as ET

# Load Job Records

### Job records is the core table.  Everything will be joined into Job Records.

##### I will only be using a small subset of the columns for demonstration purposes

In [2]:
# Loading raw sample slice
with tarfile.open('raw-sample.tar.gz', "r:*") as tar:
    # Get path to the job_records file within the tarfile
    csv_path = tar.getnames()[1]
    # Load job records file into pandas dataframe
    job_records = pd.read_csv(tar.extractfile(csv_path),
                    parse_dates = ['created','delete_date'],
                    low_memory = False,
                    usecols = ['hash','title','company_id','created','delete_date'])

# Join fs_company_reference

##### I am only joining primary information, and only based on created date as that is a good starting point.  Please reach out to Linkup if you would like to discuss options for joining to use information on multiple dates to account for jobs that span changes/M&A activity.

In [3]:
# Read PIT Ticker File
FS_reference = pd.read_csv('fs_company_reference_daily_2019-11-13.csv.gz',
                        parse_dates = ['start_date','end_date'])
                      
# Filter for only primary exchange and columns needed for join
FS_reference = FS_reference[FS_reference.primary_flag == True]

# Formatting Timestamps for Merge and dealing with missing values
FS_reference['start_date'] = FS_reference['start_date'].fillna(str(job_records.created.min().date()))
FS_reference['start_date'] = pd.to_datetime(FS_reference['start_date'])
FS_reference['end_date'] = FS_reference['end_date'].fillna(str(job_records.created.max().date()))
FS_reference['end_date'] = pd.to_datetime(FS_reference['end_date'])

# Make the db in memory
conn = sqlite3.connect(':memory:')

# Write the tables
job_records.to_sql('Job_Records', conn, index=False)
FS_reference.to_sql('FS_reference', conn, index=False)

# Query and create new joined table
qry = '''
    SELECT Job_Records.*,
    FS_reference.stock_ticker,
    FS_reference.stock_exchange_country,
    FS_reference.stock_exchange_name,
    FS_reference.primary_flag
    
    FROM Job_Records
    
    LEFT JOIN FS_reference
    ON (Job_Records.created between FS_reference.start_date and FS_reference.end_date and
       Job_Records.company_id = FS_reference.company_id);
    '''
job_records_joined = pd.read_sql_query(qry, conn)

FS_reference = None

# Join PIT Company Reference

##### For the PIT Company Reference File, most use-cases only require joining the most current company information to all records historically.  If you would like to use the PIT nature of the file, please reach out to Linkup and we would be happy to provide assistance with that.

In [4]:
PIT_Company_Reference = pd.read_csv('raw_pit_company_reference_full_2019-11-16.csv.gz')

# Filter Company Reference file to include only the companies that we pulled
#PIT_Company_Reference = PIT_Company_Reference[PIT_Company_Reference.company_id.isin(job_records.company_id.unique())]

# Filter Company Reference for the latest Information
#PIT_Company_Reference = PIT_Company_Reference[PIT_Company_Reference.end_date.isnull()]

# Make the db in memory
conn = sqlite3.connect(':memory:')

# Write the tables
job_records.to_sql('Job_Records', conn, index=False)
PIT_Company_Reference.to_sql('PIT_Company_Reference', conn, index=False)

qry = '''
    SELECT 
        Job_Records.*, 
        PIT_Company_Reference.company_url,
        PIT_Company_Reference.lei,
        PIT_Company_Reference.open_perm_id
        
    FROM Job_Records
    LEFT JOIN PIT_Company_Reference
    ON Job_Records.company_id = PIT_Company_Reference.company_id and
        Job_Records.created BETWEEN 
            PIT_Company_Reference.start_date and 
            PIT_Company_Reference.end_date
        ;
'''

job_records_joined = pd.read_sql_query(qry, conn)

PIT_Company_Reference = None

# Join Scrape Log

##### Here I will do a basic join to show the date of the next scrape change.  The primary purpose of this is so that if you see outliers, you can see when the next code change was.  If it was shortly after the outlier, the outlier is likely a scrape break vs a true signal.

In [5]:
# Load Scrape Log
Scrape_Log = pd.read_csv('raw_company_scrape_log_full_2019-11-01.csv')

# Filter Scrape Log to only include code changes for the companies that we pulled
Scrape_Log = Scrape_Log[Scrape_Log.scrape_changed == True]
Scrape_Log = Scrape_Log[Scrape_Log.company_id.isin(job_records.company_id.unique())]

In [6]:
# Shift Date for SQL query convenience
Scrape_Log['date_shifted'] = Scrape_Log.groupby(['company_id'])['date'].shift(1)
Scrape_Log['date_shifted'] = Scrape_Log['date_shifted'].fillna('2007-01-01')

Scrape_Log['date_shifted'] = pd.to_datetime(Scrape_Log['date_shifted'])
Scrape_Log['date'] = pd.to_datetime(Scrape_Log['date'])
# Make the db in memory
conn = sqlite3.connect(':memory:')

# Write the tables
job_records.to_sql('Job_Records', conn, index=False)
Scrape_Log.to_sql('Scrape_Log', conn, index=False)

# Query and create new joined table
qry = '''
    SELECT Job_Records.*,
    Scrape_Log.date as Next_Scrape_Change
    
    FROM Job_Records
    
    LEFT JOIN Scrape_Log
    ON (Job_Records.created between Scrape_Log.date_shifted and Scrape_Log.date and
       Job_Records.company_id = Scrape_Log.company_id);
    '''
job_records_joined = pd.read_sql_query(qry, conn)

Scrape_Log = None

# Join Description

In [7]:
'''
# Load Descriptions file
Job_Descriptions = pd.DataFrame(
        list(map(lambda x: (x[0].text,x[1].text),
                 ET.parse('../raw-sample/raw-sample-descriptions.xml').getroot())), 
'''    
with tarfile.open('raw-sample.tar.gz', "r:*") as tar:
    # Get path to the job_records file within the tarfile
    csv_path = tar.getnames()[0]
    # Load job records file into pandas dataframe
    #job_records = pd.read_csv(tar.extractfile(csv_path),
    #    columns = ['job_hash','description'])

    Job_Descriptions = pd.DataFrame(
        list(map(lambda x: (x[0].text,x[1].text),
                 ET.parse(tar.extractfile(csv_path)).getroot())),
        columns = ['job_hash','description'])
# Make the db in memory
conn = sqlite3.connect(':memory:')

# Write the tables
job_records.to_sql('Job_Records', conn, index=False)
Job_Descriptions.to_sql('Job_Descriptions', conn, index=False)

qry = '''
    SELECT 
        Job_Records.*, 
        Job_Descriptions.description
    
    
    FROM Job_Records
    LEFT JOIN Job_Descriptions
    ON (Job_Records.hash = Job_Descriptions.job_hash);
'''
job_records_joined = pd.read_sql_query(qry, conn)

Job_Descriptions = None

# View Results of Big Dataframe

In [8]:
job_records_joined[~job_records_joined.description.isnull()]

,hash,title,company_id,created,delete_date,description
2,0234422f9c5eff7f6d1d008c3c31dae6,"UI Artist, Double Helix Games, Amazon Game Stu...",469,2015-06-20 02:39:07+00:00,2015-09-18 07:51:10+00:00,Amazon is all in on games.\n\nWe believe the e...
7,0c392d37eb6dfb15b1ffc2a48b8d95e7,"Warehouse Team Member (Seasonal, Part Time, Fl...",469,2019-01-15 16:06:00+00:00,None,"Shifts:\n\nOver-night, Early Morning, Day, Eve..."
8,0ee6d7263a67121e8405af8ad8fdef2a,Responsable des opérations de contrôle d'inven...,469,2018-04-17 15:04:00+00:00,2018-04-18 15:05:00+00:00,ous aimez l'action ?\n\nVous aimeriez travaill...
9,0aaf97d36a5279ac18a43d1e34344232,"Relationship Manager-Newton/Waltham, MA Area",381,2015-10-15 20:58:31+00:00,2015-10-23 06:54:04+00:00,"Located in a banking center, Relationship Mana..."
13,05e060a4db459d502ed71fc78ea75f62,Database Administrator II - AMZ1781,469,2017-07-02 09:46:00+00:00,2017-07-22 19:00:00+00:00,MULTIPLE POSITIONS AVAILABLE Company: Amazon C...
...,...,...,...,...,...,...
1015682,fce9446a1c82c043b885ec53bd7c79e9,Client Service Representative,381,2017-10-10 00:03:00+00:00,2017-10-17 04:16:56+00:00,Job Description:\n\nFinancial Center Client Se...
1015683,f69454c25df4c853541eecdfc4a3d0ef,DCO Lead(Away),469,2017-11-08 16:05:00+00:00,2017-11-15 16:18:17+00:00,Are you passionate about finding process impro...
1015684,f1399ba8d1ae6adddad701beb823a8f8,Director of Inclusion and Diversity - Minneton...,383,2017-07-25 22:08:00+00:00,2017-08-29 15:50:44+00:00,"Here, you'll help attract, lead, support and r..."
1015685,f6a9ba8e6c257788e5599388f770dfd0,Registered Client Associate,381,2016-08-26 16:26:37+00:00,2016-08-30 17:50:11+00:00,Business Overview\n\nMerrill Lynch Wealth Mana...
